In [1]:
import geopandas as gpd
import pandas as pd
import folium, branca
import altair as alt
import fiona, requests, json

from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

In [2]:
print(folium.__version__)

0.7.0


In [3]:
alt.renderers.enable('jupyterlab')
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [4]:
income = pd.read_csv(r"https://raw.githubusercontent.com/pri-data/50-states/master/data/income-counties-states-national.csv", dtype={"fips":str})
income['income-2015']=pd.to_numeric(income['income-2015'], errors='coerce')

In [5]:
income.head()

,fips,county,state,income-2015,income-1989a,income-1989b,change
0,00000,US,US,55775.0,28906,53367.28102,4.316843
1,01000,Alabama,AL,44833.0,22202,40990.11877,8.571546
2,01001,Autauga County,AL,56580.0,26898,49660.04030,12.230399
3,01003,Baldwin County,AL,52387.0,24043,44389.03818,15.267074
4,01005,Barbour County,AL,31433.0,18673,34474.75398,-9.676945


In [6]:
states = gpd.GeoDataFrame.from_features(requests.get(r"https://raw.githubusercontent.com/jtbaker/data/master/geodata/usstates.geojson").json(), crs=fiona.crs.from_epsg(4326))

states.head()

,CENSUSAREA,GEO_ID,LSAD,NAME,STATE,geometry
0,30842.923,0400000US23,,Maine,23,"(POLYGON ((-67.619761 44.519754, -67.61541 44...."
1,7800.058,0400000US25,,Massachusetts,25,"(POLYGON ((-70.832044 41.606504, -70.823735 41..."
2,56538.901,0400000US26,,Michigan,26,"(POLYGON ((-88.684434 48.115785, -88.675628 48..."
3,145545.801,0400000US30,,Montana,30,"POLYGON ((-104.057698 44.997431, -104.250145 4..."
4,109781.180,0400000US32,,Nevada,32,"POLYGON ((-114.0506 37.000396, -114.049995 36...."


In [7]:
abbrs = pd.read_json(json.dumps(requests.get('https://gist.githubusercontent.com/tvpmb/4734703/raw/b54d03154c339ed3047c66fefcece4727dfc931a/US%2520State%2520List').json()))

abbrs.head(3)

,alpha-2,name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona


In [8]:
statesmerge = states.merge(abbrs,how='left', left_on='NAME', right_on='name')
statesmerge['geometry']=statesmerge.geometry.simplify(.05)

statesmerge.head()

,CENSUSAREA,GEO_ID,LSAD,NAME,STATE,geometry,alpha-2,name
0,30842.923,0400000US23,,Maine,23,"(POLYGON ((-67.619761 44.519754, -67.582113000...",ME,Maine
1,7800.058,0400000US25,,Massachusetts,25,"(POLYGON ((-70.832044 41.606504, -70.823735 41...",MA,Massachusetts
2,56538.901,0400000US26,,Michigan,26,"(POLYGON ((-88.684434 48.115785, -88.418244 48...",MI,Michigan
3,145545.801,0400000US30,,Montana,30,"POLYGON ((-104.057698 44.997431, -111.055199 4...",MT,Montana
4,109781.180,0400000US32,,Nevada,32,"POLYGON ((-114.0506 37.000396, -114.046838 36....",NV,Nevada


In [9]:
income.groupby(by="state")[['state','income-2015']].median().head()

,income-2015
state,
AK,62561.5
AL,38721.5
AR,37890.0
AZ,43810.0
CA,53341.0


In [10]:
statesmerge['medianincome']=statesmerge.merge(income.groupby(by="state")[['state','income-2015']].median(), how='left', left_on='alpha-2', right_on='state')['income-2015']
statesmerge['change']=statesmerge.merge(income.groupby(by="state")[['state','change']].median(), how='left', left_on='alpha-2', right_on='state')['change']

In [11]:
statesmerge.head()

,CENSUSAREA,GEO_ID,LSAD,NAME,STATE,geometry,alpha-2,name,medianincome,change
0,30842.923,0400000US23,,Maine,23,"(POLYGON ((-67.619761 44.519754, -67.582113000...",ME,Maine,47489.0,0.896437
1,7800.058,0400000US25,,Massachusetts,25,"(POLYGON ((-70.832044 41.606504, -70.823735 41...",MA,Massachusetts,65621.0,9.414476
2,56538.901,0400000US26,,Michigan,26,"(POLYGON ((-88.684434 48.115785, -88.418244 48...",MI,Michigan,44022.5,0.139795
3,145545.801,0400000US30,,Montana,30,"POLYGON ((-104.057698 44.997431, -111.055199 4...",MT,Montana,44998.0,14.699698
4,109781.180,0400000US32,,Nevada,32,"POLYGON ((-114.0506 37.000396, -114.046838 36....",NV,Nevada,54275.0,-1.300425


In [12]:
basechart = alt.Chart(income, title='Change in Income (%) by County').mark_bar().encode(
    x=alt.X('county:N', title='County Name', sort=alt.EncodingSortField(op='sum', field='change', order='descending')), 
    y=alt.Y('change:Q', title='Change (%)'), 
    color=alt.Color('change:Q', scale=alt.Scale(domain=[-20,0,20], range=['red', 'grey', 'green']), legend=alt.Legend(orient=alt.LegendOrient('top-right'))),
    tooltip=[alt.Tooltip('county:N',title='County'),
             alt.Tooltip('state:N', title='State'), 
             alt.Tooltip('change:Q', format=".2f", title='Change'), 
             alt.Tooltip('income-2015:Q', format="$,.2f", title='Income - 2015'),
             alt.Tooltip('income-1989b:Q', format="$,.2f", title='Income - 1989')]
).properties(width=500).interactive()

basechart

In [13]:
statesmerge['medianincome'].quantile(0.25)

44022.5

In [14]:
colormap = branca.colormap.LinearColormap(vmin=statesmerge['change'].quantile(0.0), 
                                        vmax=statesmerge['change'].quantile(1), 
                                        colors=['red','orange','lightblue','green','darkgreen'],
                                       caption="State Level Median County Household Income (%)")

In [15]:
m = folium.Map(location=[35.3, -97.6], zoom_start=4)

popup=GeoJsonPopup(fields=['NAME','change'], aliases=['State',"% Change"], localize=True, labels=True, style="background-color: yellow;")

tooltip=GeoJsonTooltip(
    fields=["NAME", "medianincome", "change"],
    aliases=["State:", "2015 Median Income(USD):", "Median % Change:"],
    localize=True,
    sticky=False,
    labels=True,
    vegalite=folium.VegaLite(basechart.to_json()),
    map_key="alpha-2",
    data_key="state",
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)


g = folium.GeoJson(
    statesmerge,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["change"])
        if x["properties"]["change"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
    popup=popup
).add_to(m)

colormap.add_to(m)

m